# So simple a child could do it

The year is 2021 and the the lockdown of Scottish primary schools is felt keenly as we enter March.
Parents across the country despair and gnash their teeth attempting to grapple with the 
schoolwork set by teachers in this new dawn of Zoom tuition.

Even before lockdown mathematics was much maligned by parents. 
How many have thrown little Johnny's homework across the room shouting: 
"Why did they change multiplication? This isn't how _I_ was taught it!" 

This sense of frustration is immortalized in Tom Leher's [New Math](https://www.youtube.com/watch?v=9mc7eb1i9o4) musical ditty from 1965, where
Leher mocks the New Math system taught in the USA fifty-six years ago. New Math didn't last, replaced once again by something else in the 70s.

The cyle will continue onwards; soon a new wave of parents will arrive to complain about how things are different.
My own feelings on this subject are we must move with the times. 
As advances in child rearing and mathematical thinking progress we must adapt how we teach, even if this removes us from our comfort zone.

Having said that, I'm about to present a problem given to a coworker's nine year old, and I do wonder what the learning outcome was supposed to _be._
Maybe this is my New Math. It's consumed more of my time than the asker had intended.

## The problem

My coworker came to me with this problem presented to his nine-year old Johnny. I am told Johnny was found to be despairing over this one for a while:

> With the numbers 1,2,3,4,5 make a 2 digit and 3 digit number (only using each number once) that gives:  
> a) the smallest product  
> b) the largest product
> 
> For example:
>
> 12 x 345 = 4140  
> 21 x 435 = 9135  
> 34 x 251 = 8534  

I sympathise, I think if someone used the word "product" around me when I was 9 I'd run for the hills.

My coworker was seeing red; as he saw it, with little guidance Johnny was expected to brute force a huge number of iterations.

The perspective of the teacher, meanwhile, was that it was not about a fixed soltuion, but finding a reasonable apporach through gut-feeling. "Anyway," she said, "you have a calculator so it should be quite fast."

Something about blindly fumbling in the darkness doesn't sit well with me; it's a joyless task. Here the magician demands you perform the trick but doesn't give any hints.

I'm not even sure how a nine-year old is supposed to have confidence in their solution. Are they expected to eventually give up and say "good enough" after enough reasonable attempts? It seems unfair. It doesn't seem a worthwhile lesson for the time-cost.

This problem is easy for a computer to solve, which I'll demonstrate below. But this isn't about a computer solving the problem, it's about humans with human brains. What I intend to do is tear this problem apart, and try to derive a set of rules human beings can follow. 

## The computer's answer

There are 120 total permutations in this question. Poor Johnny better sharpen his pencils!

The reason we can say there are 120 permutations is because at the core we're asking in how many ways can `[1, 2, 3, 4, 5]` be rearranged
without duplication, which is `5!` (5 factorial).

To illustrate this point, just image we have a list of permutations of `[1, 2, 3, 4, 5]` like so:

> 1 2 3 4 5  
> 2 1 4 3 5  
> 3 4 2 5 1  
> ...

If we introduce `x` betweem the second and third digits we now have the multiplication expression:

> 12 x 345  
> 21 x 435  
> 34 x 251  
> ...

So if there are 120 permutations of `[1, 2, 3, 4, 5]` then there must be 120 permutations of the multiplication expression.

This is a huge number for nine-year-old Johnny to work his way through. But its pretty easy for a computer, so let's get that out of the way first.


In [51]:
from itertools import permutations
from functools import reduce

# Collect the list of all possible permutations.
perms = list(permutations([1, 2, 3, 4, 5]))
print("Total number of permutations:", len(perms))

def to_product_triple(a_t, a_u, b_h, b_t, b_u):
    """
    Given: a_t, a_u, b_h, b_t, b_u, we combine the a and b 
    parts into a 2-digit and 3-digit number respectively,
    returning the resultant numbers and their product as a 
    tuple `(aa, bbb, aa * bbb)`.
    """
    aa = 10 * a_t + a_u
    bbb = 100 * b_h + 10 * b_t + b_u
    return (aa, bbb, aa * bbb)

# Collect the list of all possible products from the list of permutations.
products = [to_product_triple(*x) for x in perms]

# Finds the largest and smallest permutations and makes these the output.
fmt_perm = lambda p: "{0} x {1} = {2}".format(*p)
print("Smallest product is:", fmt_perm(reduce(lambda x, y: x if x[2] < y[2] else y, products)))
print("Largest product is:", fmt_perm(reduce(lambda x, y: x if x[2] > y[2] else y, products)))


Total number of permutations: 120
Smallest product is: 13 x 245 = 3185
Largest product is: 52 x 431 = 22412


This python-based approach is what my coworker did to solve Johnny's schoolwork. We just have to hope that Johnny's teacher wasn't expecting him to show his working.

## Reducing the problem-space and leaving brute-force behind.

Not every parent has enough Python-foo to represent the problem in computational terms.
Computer literacy is improving but it's not universal yet.

Anyway, brute-force doesn't teach us about the problem-space. If we want to better understand 
the problem, and perhaps illuminate our own understanding, we should avoid brute force where
we can. After all, we now know that `3185` is the smallest product and `22412` is the largest. But _why_? 

First, let's see if we can break the problem down from 120 permutations to something more managable.

Let's compare two permutations:

> 12 x 345 = 4140  
> 21 x 345 = 7245

Notice that the same digits occupy the two-digit number and three-digit number respectively, the only difference is the _local_ ordering of the two-digit number.

It's pretty obvious that `12 < 21`. So it also stands to reason that that `(12 x 345) < (21 x 345)`.

We can extrapolate this to the three digit number also:  `345 < 354 < 435 < 453 < 534 < 543`. Given `[3, 4, 5]` the smallest product will be a multiplication of 345, and the largest product will be a multiplication 543.

So given a number made of the digits `[1, 2]`, and another number made of the digits `[3, 4, 5]` then the smallest possible product must be `12 x 345`, and the largest product from must be `21 x 543`.

There is a general rule here:

* For the **smallest** product, the digits in a number must be sorted in **ascending** order, starting from the most significant column.
* For the **largest** product, the digits in a number must be sorted in **descending** order, starting from the most significant column.

### Calculating combinations

How does this help us? Well it lets us disregard a significant number of permutations.
In-fact there are now only 20 permutations we need to concern ourselves with now.

Consider this thought expermiment:

> Imagine you have 5 ping-pong balls labeled with the numbers 1-5 placed in a bag. 
> If you then draw two balls at random from the bag these will form your 2-digit number.
> The remaining 3 balls in the bag will form your 3-digit number. 
> 
> Now arrange the 2-balls and 3-balls to form the smallest possible permutation, then arrange the same balls to form the largest possible permutation.
>
> How many unique solutions are there?

Notice that the order in which we draw from the bag does not matter here. Drawing `1` then `2` is exactly the same as drawing `2` then `1`. We are _only_ interested in the combinations.

We can easily write out all the different combinations by hand:

```python
[1, 2], [3, 4, 5]
[1, 3], [2, 4, 5]
[1, 4], [2, 3, 5]
[1, 5], [2, 3, 4]
[2, 3], [1, 4, 5]
[2, 4], [1, 3, 5]
[2, 5], [1, 3, 4]
[3, 4], [1, 2, 5]
[3, 5], [1, 2, 4]
[4, 5], [1, 2, 3]
```

That's 10 possible combinations. So that's 10 different permutations which contains the minimum products, and 10 different permutations which contain the maximum product for a total of 20 permutations in total.

Instead of doing this by hand, there is an equation which tells us the total number of combinations: `n! / (r! * (n - r)!)`; 
where `n` is the number of balls, and `r` is the amount we draw from the bag.

Plugging `n=5` and `r=2` into the combination equation gives us `10`. Notice that `r` can be `2` _or_ `3`, it doesn't matter as the answer is still the same. 
If this surprises you, think of it like this: if we draw 2 balls from the bag, 3 will remain inside it. 
Likewise if we draw 3 balls from the bag, 2 will remain inside. It's all equivelent!

Instead of calculating all the products by hand, let's have Python do it for us.

In [53]:
from itertools import combinations

def largest_smallest_lists(list_input):
    """
    From an assumed ordered list of 5 digits `input`, returns a tuple of two lists:
    the first being the list of smallest permutations and the second beind the list 
    of largest permutations.
    """
    # List of possible two digit numbers.
    two_digits_list = list(combinations(list_input, 2))

    # List of possible 3-digit numbers that are compliments to the 2-digit numbers.
    three_digits_list = [[y for y in list_input if y not in digits] for digits in two_digits_list]

    # Calculate the smallest possible products from the combination of the 2-digit and 3-digit number.
    # Note: the number digits are already sorted in ascending order as `combinations` guarantees ordering is preserved.
    smallest_list = [to_product_triple(*x, *y) for (x, y) in zip(two_digits_list, three_digits_list)]

    # Calculate the largest possible products from the combination of the 2-digit and 3-digit number.
    # Note: we reverse the digit order of the 2-digits and 3-digit numbers to achieve this.
    largest_list = [to_product_triple(*x[::-1], *y[::-1]) for (x, y) in zip(two_digits_list, three_digits_list)]
    
    return(smallest_list, largest_list)

inputs = [1, 2, 3, 4, 5]
largest_list, smallest_list = largest_smallest_lists(inputs)

# Show our calculations.
print("{0:20} {1}".format("Smallest products", "Largest products"))
for (s, l) in zip(largest_list, smallest_list):
    print("{0:20} {1}".format(fmt_perm(s), fmt_perm(l)))

Smallest products    Largest products
12 x 345 = 4140      21 x 543 = 11403
13 x 245 = 3185      31 x 542 = 16802
14 x 235 = 3290      41 x 532 = 21812
15 x 234 = 3510      51 x 432 = 22032
23 x 145 = 3335      32 x 541 = 17312
24 x 135 = 3240      42 x 531 = 22302
25 x 134 = 3350      52 x 431 = 22412
34 x 125 = 4250      43 x 521 = 22403
35 x 124 = 4340      53 x 421 = 22313
45 x 123 = 5535      54 x 321 = 17334


20 is a tad more managable than 120! We can even filter these lists just by eyeballing them.

Look closely at the list of smallest products: we can see the product values are clearly increasing. _Except_ in the case of the first entry. 
What makes the first entry different?


## Beyond [1, 2, 3, 4, 5]

If we want to understand 




In [50]:
all_inputs = combinations([1, 2, 3, 4, 5, 6, 7, 8, 9], 5)

print("{0:20} {1:20} {2}".format("Inputs", "Smallest products", "Largest products"))

for input_perm in all_inputs:    
    small_list, large_list = largest_smallest_lists(input_perm)
    
    smallest = reduce(lambda x, y: x if x[2] < y[2] else y, small_list)
    largest= reduce(lambda x, y: x if x[2] > y[2] else y, large_list)

    print("{0:20} {1:20} {2}".format(str(input_perm), fmt_perm(smallest), fmt_perm(largest)))

Inputs               Smallest products    Largest products
(1, 2, 3, 4, 5)      13 x 245 = 3185      52 x 431 = 22412
(1, 2, 3, 4, 6)      13 x 246 = 3198      62 x 431 = 26722
(1, 2, 3, 4, 7)      13 x 247 = 3211      72 x 431 = 31032
(1, 2, 3, 4, 8)      13 x 248 = 3224      82 x 431 = 35342
(1, 2, 3, 4, 9)      13 x 249 = 3237      92 x 431 = 39652
(1, 2, 3, 5, 6)      13 x 256 = 3328      62 x 531 = 32922
(1, 2, 3, 5, 7)      13 x 257 = 3341      72 x 531 = 38232
(1, 2, 3, 5, 8)      13 x 258 = 3354      82 x 531 = 43542
(1, 2, 3, 5, 9)      13 x 259 = 3367      92 x 531 = 48852
(1, 2, 3, 6, 7)      13 x 267 = 3471      72 x 631 = 45432
(1, 2, 3, 6, 8)      13 x 268 = 3484      82 x 631 = 51742
(1, 2, 3, 6, 9)      13 x 269 = 3497      92 x 631 = 58052
(1, 2, 3, 7, 8)      13 x 278 = 3614      82 x 731 = 59942
(1, 2, 3, 7, 9)      13 x 279 = 3627      92 x 731 = 67252
(1, 2, 3, 8, 9)      13 x 289 = 3757      92 x 831 = 76452
(1, 2, 4, 5, 6)      14 x 256 = 3584      62 x 541 = 335